In [1]:
import json
import re
import time
import requests
import numpy as np
import pandas as pd
import os
import glob
from tqdm import tqdm
import datetime as dt
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import warnings
warnings.filterwarnings('ignore')

In [2]:
date_now = dt.datetime.now().date().strftime('%d.%m.%Y')
way = os.getcwd()

# Cookies and Headers для парсинга скидки и цены

In [3]:
cookies_discount = {
    'tmr_lvid': '286726132090325c59b36aa6b5e4aeaa',
    'tmr_lvidTS': '1692163620235',
    '_ym_uid': '1692163620496112993',
    '_ym_d': '1692163620',
    'city_id': '118',
    'city_name': '%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3',
    'city_full_name': '%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3',
    'region_id': '090a0c31-3b91-49a3-aa35-0c784898170b',
    'region_name': '%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3',
    'region_subdomain': 'spb',
    '_tt_enable_cookie': '1',
    '_ttp': 'tbKEEA3IkGrgdKpaT6-HRNf0K8h',
    'ccart': 'off',
    'uxs_uid': '898ab7e0-3bf5-11ee-8efe-f5b770db12a1',
    'loggedin': '%D0%AE%D1%80%D0%B8%D0%B9',
    'cseid': 'f4935df6bdac0029832e99cef94a5bc2',
    'sessionid': 'nr24mlml3ze25rk734gkkmgx3yv9ou7x',
    'authorization': 'true',
    'mindboxDeviceUUID': 'c254fa3f-cfcd-4146-9778-65be21c6a4df',
    'directCrm-session': '%7B%22deviceGuid%22%3A%22c254fa3f-cfcd-4146-9778-65be21c6a4df%22%7D',
    'auid': '651660ce-a06e-4b17-bd6f-69feaf135b08:1r5lu8:WYtTZugo2xVE6SqAc5ZSp_W-n1s5ztPHYxPs8YIwGQA',
    '_ga_G74SVHM1ZG': 'GS1.2.1700824600.19.0.1700824600.60.0.0',
    '_ga_GRDJMMMRHQ': 'GS1.2.1700824600.33.0.1700824601.59.0.0',
    '_ga_GRSMPDQ6SE': 'GS1.2.1700824598.88.1.1700825516.60.0.0',
    '_ga_M9ZHPEJXBM': 'GS1.2.1700824599.88.1.1700825517.60.0.0',
    'qr-go-to-the-app-banner': '1',
    'sncid': 'b36ee46b-981d-11e5-add2-0050568f2465:1rBa43:O1Wy8KuwGDddSCM7NMHFsjGhdPaqxcjhdblvWWbVams',
    'clt': 'new_old',
    '_ym_isad': '2',
    '_gid': 'GA1.2.2131018395.1702274408',
    'city_auto_popup_shown': '1',
    '_ym_visorc': 'b',
    '_ga': 'GA1.2.333284150.1692163620',
    '_ga_HJNSJ6NG5J': 'GS1.1.1702276831.120.1.1702280271.20.0.0',
    'digi_uc': 'W1sidiIsIjM0OTMxNCIsMTcwMjI4MTIxMjA0Nl0sWyJ2IiwiODQwMjIiLDE3MDIyODAyMzEzNjZdLFsidiIsIjQ2Nzk3IiwxNzAyMjgwMTI4Mzc3XSxbInYiLCI5OTM3NTQiLDE3MDIyNzgwMTczMjddLFsidiIsIjE0NjUyMSIsMTcwMjI3NzI5NDExM10sWyJ2IiwiOTI2ODAiLDE3MDIyNzY5NDQ1NDRdLFsidiIsIjg4MjEzIiwxNzAxODU5MjE3OTg0XSxbInYiLCI5MTAyMSIsMTcwMTg1OTA2ODA2MF0sWyJ2IiwiOTg3MDUwIiwxNzAxNzc4NDc0NzY5XSxbInYiLCIyNTYyMTciLDE3MDEzNDIzNTc4NTRdXQ==',
}

headers_discount = {
    'AB-TESTS': '{}',
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
    'Connection': 'keep-alive',
    'Content-Type': 'application/json',
    # 'Cookie': 'tmr_lvid=286726132090325c59b36aa6b5e4aeaa; tmr_lvidTS=1692163620235; _ym_uid=1692163620496112993; _ym_d=1692163620; city_id=118; city_name=%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3; city_full_name=%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3; region_id=090a0c31-3b91-49a3-aa35-0c784898170b; region_name=%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3; region_subdomain=spb; _tt_enable_cookie=1; _ttp=tbKEEA3IkGrgdKpaT6-HRNf0K8h; ccart=off; uxs_uid=898ab7e0-3bf5-11ee-8efe-f5b770db12a1; loggedin=%D0%AE%D1%80%D0%B8%D0%B9; cseid=f4935df6bdac0029832e99cef94a5bc2; sessionid=nr24mlml3ze25rk734gkkmgx3yv9ou7x; authorization=true; mindboxDeviceUUID=c254fa3f-cfcd-4146-9778-65be21c6a4df; directCrm-session=%7B%22deviceGuid%22%3A%22c254fa3f-cfcd-4146-9778-65be21c6a4df%22%7D; auid=651660ce-a06e-4b17-bd6f-69feaf135b08:1r5lu8:WYtTZugo2xVE6SqAc5ZSp_W-n1s5ztPHYxPs8YIwGQA; _ga_G74SVHM1ZG=GS1.2.1700824600.19.0.1700824600.60.0.0; _ga_GRDJMMMRHQ=GS1.2.1700824600.33.0.1700824601.59.0.0; _ga_GRSMPDQ6SE=GS1.2.1700824598.88.1.1700825516.60.0.0; _ga_M9ZHPEJXBM=GS1.2.1700824599.88.1.1700825517.60.0.0; qr-go-to-the-app-banner=1; sncid=b36ee46b-981d-11e5-add2-0050568f2465:1rBa43:O1Wy8KuwGDddSCM7NMHFsjGhdPaqxcjhdblvWWbVams; clt=new_old; _ym_isad=2; _gid=GA1.2.2131018395.1702274408; city_auto_popup_shown=1; _ym_visorc=b; _ga=GA1.2.333284150.1692163620; _ga_HJNSJ6NG5J=GS1.1.1702276831.120.1.1702280271.20.0.0; digi_uc=W1sidiIsIjM0OTMxNCIsMTcwMjI4MTIxMjA0Nl0sWyJ2IiwiODQwMjIiLDE3MDIyODAyMzEzNjZdLFsidiIsIjQ2Nzk3IiwxNzAyMjgwMTI4Mzc3XSxbInYiLCI5OTM3NTQiLDE3MDIyNzgwMTczMjddLFsidiIsIjE0NjUyMSIsMTcwMjI3NzI5NDExM10sWyJ2IiwiOTI2ODAiLDE3MDIyNzY5NDQ1NDRdLFsidiIsIjg4MjEzIiwxNzAxODU5MjE3OTg0XSxbInYiLCI5MTAyMSIsMTcwMTg1OTA2ODA2MF0sWyJ2IiwiOTg3MDUwIiwxNzAxNzc4NDc0NzY5XSxbInYiLCIyNTYyMTciLDE3MDEzNDIzNTc4NTRdXQ==',
    'Origin': 'https://spb.sunlight.net',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-site',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'X-Requested-With': 'SunlightFrontendApp',
    'sec-ch-ua': '"Not_A Brand";v="8", "Chromium";v="120", "Google Chrome";v="120"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
}

# params = {
#     'article': '35517',
#     'city_id': '118',
# }

# response = requests.get(
#     'https://api.sunlight.net/v6/catalog/products/card/purchase/',
#     params=params,
#     cookies=cookies_discount,
#     headers=headers_discount,
# )

# Сбор скидок и цен

In [4]:
def get_personal_price(table):
    data_price = []
    for art in tqdm(range(len(table['article']))):
        params = {
            'article': table['article'][art],
            'city_id': '118'
        }
        try:
            price_json = requests.get(
                'https://api.sunlight.net/v6/catalog/products/card/prices/',
                params=params,
                cookies=cookies_discount,
                headers=headers_discount,
                timeout=15,
                verify=False
        )
        except:
            time.sleep(6)
            price_json = requests.get(
                'https://api.sunlight.net/v6/catalog/products/card/prices/',
                params=params,
                cookies=cookies_discount,
                headers=headers_discount,
                timeout=15,
                verify=False
                            )
        try:
            discount_json = requests.get(
                'https://api.sunlight.net/v6/catalog/products/card/purchase/',
                params=params,
                cookies=cookies_discount,
                headers=headers_discount,
                timeout=15,
                verify=False
                            )
        except:
            time.sleep(6)
            discount_json = requests.get(
                'https://api.sunlight.net/v6/catalog/products/card/purchase/',
                params=params,
                cookies=cookies_discount,
                headers=headers_discount,
                timeout=15,
                verify=False
                            )
        discount = []
        try:
            price_old = price_json.json()['content']['default_price']['base_sum']
        except:
            price_old = np.nan
        try:
            price = price_json.json()['content']['default_price'][('result_sum')]
        except:
            price = np.nan
        try:
            price_personal = price_json.json()['content']['default_price']['personal_block']['value']
        except:
            price_personal = np.nan

        try:
            for i in range(len(discount_json.json()['content']['nameplates'])):
                discount.append(discount_json.json()['content']['nameplates'][i]['text']['label'])
        except:
            discount.append(np.nan)
            
        data_price.append({
            'article': table['article'][art],
            'price_old': price_old,
            'price': price,
            'price_personal': price_personal,
            'discount': discount

        })
    return data_price

# Сбор цены, названия, группы, артикула

In [5]:
def get_sunlight(soup):
    try:
        price = soup.find(class_="supreme-product-card__price-discount-price").text
        price = re.sub(r'\D', '', str(price))
    except:
        price = np.nan
    try:
        name_product = soup.find('h1', class_="supreme-product-card__info-name-title").text
    except:
        name_product = np.nan
    try:
        group_product = soup.find_all('span', {'property': 'name'})[1].text
    except:
        group_product = np.nan
    try:
        article_product = soup.find_all('div', class_="supreme-product-card__info-name-article")[0].find('span').text
        article_product = re.sub('\D', '', str(article_product))
    except:
        article_product = np.nan
    return price, name_product, group_product, article_product

# Сбор всех параметров словарем

In [6]:
def get_params(soup):
    names = []
    values = []
    leight = len(soup.find_all('span', {'class': 'supreme-product-card-description__item-text'}))
    list_names = soup.find_all('div', {'class': 'supreme-product-card-description__item-title'})[-leight:]
    for i in range(leight):
        try:
            nam = list_names[i].text
            names.append(nam.replace('\n', '').strip())
        except:
            names.append(np.nan)
        try:
            values.append(soup.find_all('span', {'class': 'supreme-product-card-description__item-text'})[i].text)
        except:
            values.append(np.nan)
    sunlight_params = dict(zip(names, values))
    return sunlight_params

# Сбор ссылок

In [7]:
def get_links(url_sunlight):
    home = 'https://sunlight.net'
    links = []
    pages_max = 63
    
    pages = int(pages_max + 1)
    for i in tqdm(range(1, pages)):
        url = f'{url_sunlight}&page={i}'
        resp = requests.get(url, headers={'User-Agent': UserAgent().chrome}, verify=False)
        soup = BeautifulSoup(resp.text, 'html.parser')
        for link in soup.find_all('div', class_="cl-item js-cl-item"):
            links.append(home + link.find('a', href=True)['href'])
    links = list(map(lambda x: x.replace('#product-reviews', ''), links))
    return links

# Общий сбор данных

In [8]:
def sunlight_parser(links):
    data = []
    click = 0
    for link_pars in tqdm(links):
        resp = requests.get(link_pars, headers={'User-Agent': UserAgent().chrome}, timeout=70, verify=False)
        good = BeautifulSoup(resp.text, 'lxml')
        data.append({"date": date_now,
                    "competitor": "Sunlight",
                    "name": get_sunlight(good)[1],
                    "group": get_sunlight(good)[2],
                    "params": get_params(good),
                    "price": get_sunlight(good)[0],
                    "article": get_sunlight(good)[3],
                    'url': link_pars
                    })
        if click == 500:
            time.sleep(2)
        else:
            continue
    data = pd.DataFrame(data)
    return data

In [9]:
links_sunlight = get_links('https://spb.sunlight.net/catalog/?proba=2494&stone=1')

  0%|          | 0/63 [00:00<?, ?it/s]

100%|██████████| 63/63 [00:57<00:00,  1.09it/s]


In [10]:
links = pd.DataFrame(links_sunlight)

In [11]:
links = links.drop_duplicates()

In [12]:
links = pd.DataFrame()

In [13]:
links.to_csv(f'{way}\links.csv', index = False)

In [14]:
slices = 10
chunks = np.array_split(links_sunlight, slices)

In [15]:
n = 0
for i in range(n, slices):
    n += 1
    print(f'начало парсинга {n} части ссылок')
    df = sunlight_parser(chunks[i])
    df.to_csv(rf'{way}\parsed_data\bk_sunlight_{date_now}_{n}.csv', index=False)
    time.sleep(5)

начало парсинга 1 части ссылок


100%|██████████| 376/376 [05:18<00:00,  1.18it/s]


начало парсинга 2 части ссылок


100%|██████████| 376/376 [05:18<00:00,  1.18it/s]


начало парсинга 3 части ссылок


...

100%|██████████| 376/376 [05:42<00:00,  1.10it/s]


начало парсинга 9 части ссылок


100%|██████████| 376/376 [05:42<00:00,  1.10it/s]


начало парсинга 10 части ссылок


100%|██████████| 375/375 [03:20<00:00,  1.87it/s]


In [16]:
os.chdir(rf'{way}\parsed_data')
extension = 'csv'
all_filenames = [i for i in glob.glob('bk_sunlight_*.{}'.format(extension))]
all_filenames

['bk_sunlight_28.06.2024_1.csv',
 'bk_sunlight_28.06.2024_10.csv',
 'bk_sunlight_28.06.2024_2.csv',
 'bk_sunlight_28.06.2024_3.csv',
 'bk_sunlight_28.06.2024_4.csv',
 'bk_sunlight_28.06.2024_5.csv',
 'bk_sunlight_28.06.2024_6.csv',
 'bk_sunlight_28.06.2024_7.csv',
 'bk_sunlight_28.06.2024_8.csv',
 'bk_sunlight_28.06.2024_9.csv']

In [17]:
data = pd.DataFrame()
for name in all_filenames:
    df = pd.read_csv(name)
    data = pd.concat([data, df], ignore_index= True)

In [18]:
data = data.dropna().reset_index()
data['article'] = data['article'].astype(int)

In [19]:
data

,index,date,competitor,name,group,params,price,article,url
0,0,28.06.2024,Sunlight,\n Золотые серьги\n,Серьги,"{'Модель': '200100.16-1ш/з*', 'Материал издели...",10490,271510,https://sunlight.net/catalog/earring_271510.html
1,1,28.06.2024,Sunlight,\n Золотые серьги\n,Серьги,"{'Модель': '200200.4-1ш/з*', 'Материал изделия...",9990,271409,https://sunlight.net/catalog/earring_271409.html
2,2,28.06.2024,Sunlight,\n Золотые серьги\n,Серьги,"{'Модель': '10-00-0000-27628*', 'Материал изде...",20990,235978,https://sunlight.net/catalog/earring_235978.html
3,3,28.06.2024,Sunlight,\n Золотой крест-подвеска\n,Крест-подвеска,"{'Модель': '10021700*', 'Материал изделия': 'Р...",7990,74404,https://sunlight.net/catalog/pendants_74404.html
4,4,28.06.2024,Sunlight,\n Золотое шейное украшение\n,Шейное украшение,"{'Модель': '6030408-40*', 'Материал изделия': ...",15990,334031,https://sunlight.net/catalog/necklace_119474.html
...,...,...,...,...,...,...,...,...,...
3754,3754,28.06.2024,Sunlight,\n Золотой браслет\n,Браслеты,"{'Модель': '46-0015-11*', 'Материал изделия': ...",48060,345091,https://sunlight.net/catalog/bracelets_345091....
3755,3755,28.06.2024,Sunlight,\n Золотой браслет\n,Браслеты,"{'Модель': '46-0009-11-61*', 'Материал изделия...",81010,345089,https://sunlight.net/catalog/bracelets_345089....
3756,3756,28.06.2024,Sunlight,\n Золотой браслет\n,Браслеты,"{'Модель': '46-0002-11-61*', 'Материал изделия...",53860,345086,https://sunlight.net/catalog/bracelets_345086....
3757,3757,28.06.2024,Sunlight,\n Золотой браслет\n,Браслеты,"{'Модель': '442-00-0180-31130*', 'Материал изд...",298327,344752,https://sunlight.net/catalog/bracelets_344752....


In [20]:
data_discount = get_personal_price(data)
data_discount = pd.DataFrame(data_discount)

100%|██████████| 3759/3759 [41:19<00:00,  1.52it/s]  


In [21]:
data_discount.sample(5)

,article,price_old,price,price_personal,discount
1887,320291,51 980 ₽,25 990 ₽,15 594 ₽,"[Ещё -2 000₽, код: ЛЕТО до 1 июля, Цены 8 март..."
1391,344957,47 980 ₽,23 990 ₽,14 394 ₽,"[Ещё -2 000₽, код: ЛЕТО до 1 июля, Цены 8 март..."
294,301207,51 980 ₽,25 990 ₽,15 594 ₽,"[Ещё -5 000₽, код: ЛЕТО до 1 июля, Цены 8 март..."
1098,89763,,17 126 ₽,NaN,[ХИТ-цена]
787,301761,149 980 ₽,74 990 ₽,44 994 ₽,"[Ещё -10 000₽, код: ЛЕТО до 1 июля, Цены 8 мар..."


In [22]:
data = data.merge(data_discount, on='article', how='left')

In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5559 entries, 0 to 5558
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   index           5559 non-null   int64 
 1   date            5559 non-null   object
 2   competitor      5559 non-null   object
 3   name            5559 non-null   object
 4   group           5559 non-null   object
 5   params          5559 non-null   object
 6   price_x         5559 non-null   int64 
 7   article         5559 non-null   int32 
 8   url             5559 non-null   object
 9   price_old       5559 non-null   object
 10  price_y         5559 non-null   object
 11  price_personal  3782 non-null   object
 12  discount        5559 non-null   object
dtypes: int32(1), int64(2), object(10)
memory usage: 543.0+ KB


In [24]:
data['discount'].value_counts()

discount
[ХИТ-цена]                                                                          1655
[Ещё -1 000₽, код: ЛЕТО до 1 июля, Цены 8 марта! Ещё -10%, код: МАРТ до 23:59]      1034
[Ещё -2 000₽, код: ЛЕТО до 1 июля, Цены 8 марта! Ещё -10%, код: МАРТ до 23:59]       566
[Ещё -500₽, код: ЛЕТО до 1 июля, Цены 8 марта! Ещё -10%, код: МАРТ до 23:59]         478
[Ещё -3 500₽, код: ЛЕТО до 1 июля, Цены 8 марта! Ещё -10%, код: МАРТ до 23:59]       273
[Ещё -2 000₽, код: ЛЕТО до 1 июля, Цены 8 марта! Ещё -5%, код: МАРТ до 23:59]        196
[Ещё -5 000₽, код: ЛЕТО до 1 июля, Цены 8 марта! Ещё -10%, код: МАРТ до 23:59]       170
[Ещё -15%, код: ВМЕСТЕ при покупке от 2-х]                                           124
[ХИТ-цена, Выгодно! Цепи 4990 р/г]                                                   120
[Ещё -1 000₽, код: ЛЕТО до 1 июля, Цены 8 марта! Ещё -15%, код: МАРТ до 23:59]        86
[Ещё -1 000₽, код: ЛЕТО до 1 июля, Цены 8 марта! Ещё -5%, код: МАРТ до 23:59]         63
[Ещё -200₽, 

In [25]:
data.head(20)

,index,date,competitor,name,group,params,price_x,article,url,price_old,price_y,price_personal,discount
0,0,28.06.2024,Sunlight,\n Золотые серьги\n,Серьги,"{'Модель': '200100.16-1ш/з*', 'Материал издели...",10490,271510,https://sunlight.net/catalog/earring_271510.html,,10 490 ₽,NaN,[ХИТ-цена]
1,0,28.06.2024,Sunlight,\n Золотые серьги\n,Серьги,"{'Модель': '200100.16-1ш/з*', 'Материал издели...",10490,271510,https://sunlight.net/catalog/earring_271510.html,,10 490 ₽,NaN,[ХИТ-цена]
2,0,28.06.2024,Sunlight,\n Золотые серьги\n,Серьги,"{'Модель': '200100.16-1ш/з*', 'Материал издели...",10490,271510,https://sunlight.net/catalog/earring_271510.html,,10 490 ₽,NaN,[ХИТ-цена]
3,0,28.06.2024,Sunlight,\n Золотые серьги\n,Серьги,"{'Модель': '200100.16-1ш/з*', 'Материал издели...",10490,271510,https://sunlight.net/catalog/earring_271510.html,,10 490 ₽,NaN,[ХИТ-цена]
4,0,28.06.2024,Sunlight,\n Золотые серьги\n,Серьги,"{'Модель': '200100.16-1ш/з*', 'Материал издели...",10490,271510,https://sunlight.net/catalog/earring_271510.html,,10 490 ₽,NaN,[ХИТ-цена]
5,0,28.06.2024,Sunlight,\n Золотые серьги\n,Серьги,"{'Модель': '200100.16-1ш/з*', 'Материал издели...",10490,271510,https://sunlight.net/catalog/earring_271510.html,,10 490 ₽,NaN,[ХИТ-цена]
6,1,28.06.2024,Sunlight,\n Золотые серьги\n,Серьги,"{'Модель': '200200.4-1ш/з*', 'Материал изделия...",9990,271409,https://sunlight.net/catalog/earring_271409.html,,9 990 ₽,NaN,[ХИТ-цена]
7,1,28.06.2024,Sunlight,\n Золотые серьги\n,Серьги,"{'Модель': '200200.4-1ш/з*', 'Материал изделия...",9990,271409,https://sunlight.net/catalog/earring_271409.html,,9 990 ₽,NaN,[ХИТ-цена]
8,1,28.06.2024,Sunlight,\n Золотые серьги\n,Серьги,"{'Модель': '200200.4-1ш/з*', 'Материал изделия...",9990,271409,https://sunlight.net/catalog/earring_271409.html,,9 990 ₽,NaN,[ХИТ-цена]
9,1,28.06.2024,Sunlight,\n Золотые серьги\n,Серьги,"{'Модель': '200200.4-1ш/з*', 'Материал изделия...",9990,271409,https://sunlight.net/catalog/earring_271409.html,,9 990 ₽,NaN,[ХИТ-цена]


In [26]:
data.to_csv(rf'{way}\parsed_data\result\bk_sunlight_{date_now}.csv', index=False)
data.to_csv(r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Тренкин Сергей\Мониторинг Sokolov и Sunlight\Данные парсинга\sklv_sl_data\raw_data\\' + f'bk_sunlight_{date_now}.csv', index=False)

Для удаления файлов внутри папки:

In [27]:
def delete_files_in_folder(folder_path):
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        try:
            if os.path.isfile(file_path):
                os.remove(file_path)
        except Exception as e:
            print(f'Ошибка при удалении файла {file_path}. {e}')

In [28]:
delete_files_in_folder(f'{way}\parsed_data')

In [29]:
# !jupyter nbconvert --to script sunlight_bk_cb.ipynb